In [1]:
import xarray as xr
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from dask.diagnostics import ProgressBar
import matplotlib.colors as colors
import datetime as dt

In [2]:
rb = plt.cm.RdBu
bm = plt.cm.Blues

In [3]:
import os
cwd = os.getcwd()

cwd = cwd + '/'
cwd

'/home/563/ab2313/MJO/'

In [4]:
save_dir = cwd + 'extreme_plots/data/'

In [5]:
def sel_sig_all(normal, boot):
    # ! Potential mistake: I only have boot 5th and 95th greater than 1. I should probably have the 
    # actual value as 1. Not sure if this will make a difference though. Might just leave it. Answer:
    # when you did the old one it didn't chnage anything. Still just got pos and negs on either side.


    # Both 5th and 95th percentile >= 1 (from bootdata)
    station_sig_pos = normal.precip.where(
        np.logical_and(
            boot.isel(percentile = 0).precip >= 1, 
            boot.isel(percentile = 1).precip >= 1)
            )
    

    # Both 5th and 95th percentile <= 1
    station_sig_neg = normal.precip.where(
        np.logical_and(
            boot.isel(percentile = 0).precip <= 1, 
            boot.isel(percentile = 1).precip <= 1)
        )
    
    # Using scotts code to combine this into one array:
        # If the pos is finite then use the value from pos, else use the value form negative
    total = xr.where(np.isfinite(station_sig_pos),station_sig_pos, station_sig_neg)
    
    # Reutrns as a dataset otherwise, want a datarray
    total = total.to_dataset(name = 'precip')
    
    return total


# Count

In [6]:
name = 'count'


# Reading in the data for this specific index
directory1 = cwd +'AWAP/' + name +'_data/'
tail1 = ''


directory2 = cwd + 'w_ENSO/'  +name +'_data/'
tail2 = '_nino'
directory1,directory2 

('/home/563/ab2313/MJO/AWAP/count_data/',
 '/home/563/ab2313/MJO/w_ENSO/count_data/')

## Normal

In [59]:
'''------Normal------'''


'''Normal data'''
# path = directory1 + name+ '_50' + tail1 +'.nc'
# data_50 = xr.open_dataset(path)
# data_50 = data_50.drop('nino')
    
path = directory1 + name + '_90' + tail1 +'.nc'
data_90 = xr.open_dataset(path)


path = directory1  + name+ '_95' + tail1 +'.nc'
data_95 = xr.open_dataset(path)



'''Bootstrapped Data'''
# path =directory1  + name+ '_boot_50' + tail1 +'.nc'
# bootdata_50= xr.open_dataset(path)


path =directory1  + name+ '_boot_90' + tail1 +'.nc'
bootdata_90= xr.open_dataset(path)



path = directory1  + name + '_boot_95' + tail1 +'.nc'
bootdata_95 = xr.open_dataset(path)


In [60]:
count_data= xr.concat([ data_90, data_95], pd.Index(['90','95'], name  = 'percentile'))
count_data.to_netcdf(save_dir + 'count_data.nc')

In [52]:

sig90 = sel_sig_all(data_90, bootdata_90) # 90th Percentile


sig95 = sel_sig_all(data_95, bootdata_95) # 95th Percentile

In [ ]:
count_sig = xr.concat([sig90, sig95], pd.Index(['90','95'], name  = 'percentile'))

In [56]:
count_sig

<xarray.Dataset>
Dimensions:     (lat: 53, lon: 178, mjo: 3, month: 6, percentile: 2)
Coordinates:
  * lat         (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
  * lon         (lon) float64 112.0 112.2 112.5 112.8 ... 155.8 156.0 156.2
  * month       (month) int64 10 11 12 1 2 3
  * mjo         (mjo) object 'enhanced' 'suppressed' 'inactive'
  * percentile  (percentile) object '90' '95'
Data variables:
    precip      (percentile, mjo, month, lat, lon) float64 nan nan ... nan nan

In [54]:
count_sig.to_netcdf(save_dir + 'count_sig.nc')

## ENSO

In [7]:
'''------ENSO------'''


'''Normal data'''
path = directory2 + name+ '_50' + tail2 +'.nc'
data_50 = xr.open_dataset(path)
data_50 = data_50.drop('nino')
    
path = directory2 + name + '_90' + tail2 +'.nc'
data_90 = xr.open_dataset(path)
data_90 = data_90.drop('nino')

path = directory2  + name+ '_95' + tail2 +'.nc'
data_95 = xr.open_dataset(path)
data_95 = data_95.drop('nino')


'''Bootstrapped Data'''
path =directory2  + name+ '_boot_50' + tail2 +'.nc'
bootdata_50= xr.open_dataset(path)


path =directory2  + name+ '_boot_90' + tail2 +'.nc'
bootdata_90= xr.open_dataset(path)



path = directory2  + name + '_boot_95' + tail2 +'.nc'
bootdata_95 = xr.open_dataset(path)
# bootdata_95 = bootdata_95.rename({'nino':'enso'})

In [9]:
count_data_enso= xr.concat([data_50, data_90, data_95], pd.Index(['50','90','95'], name  = 'percentile'))
count_data_enso.to_netcdf(save_dir + 'count_data_enso.nc')

In [10]:
sig50 = sel_sig_all(data_50, bootdata_50) # 50th Percentile

sig90 = sel_sig_all(data_90, bootdata_90) # 90th Percentile

sig95 = sel_sig_all(data_95, bootdata_95) # 95th Percentile

In [11]:
count_sig_enso = xr.concat([sig50, sig90, sig95], pd.Index(['50','90','95'], name  = 'percentile'))

count_sig_enso.to_netcdf(save_dir + 'count_sig_enso.nc')

# Intensity

In [65]:
name = 'int'


# Reading in the data for this specific index
directory1 = cwd +'AWAP/' + name +'_data/'
tail1 = ''


directory2 = cwd + 'w_ENSO/'  +name +'_data/'
tail2 = '_nino'
directory1,directory2 

('/home/563/ab2313/MJO/AWAP/int_data/',
 '/home/563/ab2313/MJO/w_ENSO/int_data/')

## Normal

In [66]:
'''------Normal------'''


'''Normal data'''
# path = directory1 + name+ '_50' + tail1 +'.nc'
# data_50 = xr.open_dataset(path)
# data_50 = data_50.drop('nino')
    
path = directory1 + name + '_90' + tail1 +'.nc'
data_90 = xr.open_dataset(path)


path = directory1  + name+ '_95' + tail1 +'.nc'
data_95 = xr.open_dataset(path)



'''Bootstrapped Data'''
# path =directory1  + name+ '_boot_50' + tail1 +'.nc'
# bootdata_50= xr.open_dataset(path)


path =directory1  + name+ '_boot_90' + tail1 +'.nc'
bootdata_90= xr.open_dataset(path)



path = directory1  + name + '_boot_95' + tail1 +'.nc'
bootdata_95 = xr.open_dataset(path)


In [67]:
int_data= xr.concat([ data_90, data_95], pd.Index(['90','95'], name  = 'percentile'))
int_data.to_netcdf(save_dir + 'int_data.nc')

In [68]:

sig90 = sel_sig_all(data_90, bootdata_90) # 90th Percentile


sig95 = sel_sig_all(data_95, bootdata_95) # 95th Percentile

In [70]:
int_sig

<xarray.Dataset>
Dimensions:     (lat: 53, lon: 178, mjo: 3, month: 6, percentile: 2)
Coordinates:
  * lat         (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
  * lon         (lon) float64 112.0 112.2 112.5 112.8 ... 155.8 156.0 156.2
  * month       (month) int64 10 11 12 1 2 3
  * mjo         (mjo) object 'enhanced' 'suppressed' 'inactive'
  * percentile  (percentile) object '90' '95'
Data variables:
    precip      (percentile, mjo, month, lat, lon) float64 nan nan ... nan nan

In [69]:
int_sig = xr.concat([sig90, sig95], pd.Index(['90','95'], name  = 'percentile'))

In [71]:
int_sig.to_netcdf(save_dir + 'int_sig.nc')

# Prop

In [72]:
name = 'prop'


# Reading in the data for this specific index
directory1 = cwd +'AWAP/' + name +'_data/'
tail1 = ''


directory2 = cwd + 'w_ENSO/'  +name +'_data/'
tail2 = '_nino'
directory1,directory2 

('/home/563/ab2313/MJO/AWAP/prop_data/',
 '/home/563/ab2313/MJO/w_ENSO/prop_data/')

In [74]:
'''------Normal------'''


'''Normal data'''
# path = directory1 + name+ '_50' + tail1 +'.nc'
# data_50 = xr.open_dataset(path)
# data_50 = data_50.drop('nino')
    
path = directory1 + name + '_90' + tail1 +'.nc'
data_90 = xr.open_dataset(path)


path = directory1  + name+ '_95' + tail1 +'.nc'
data_95 = xr.open_dataset(path)



'''Bootstrapped Data'''
# path =directory1  + name+ '_boot_50' + tail1 +'.nc'
# bootdata_50= xr.open_dataset(path)


path =directory1  + name+ '_boot_90' + tail1 +'.nc'
bootdata_90= xr.open_dataset(path)



path = directory1  + name + '_boot_95' + tail1 +'.nc'
bootdata_95 = xr.open_dataset(path)

In [75]:
prop_data= xr.concat([ data_90, data_95], pd.Index(['90','95'], name  = 'percentile'))
prop_data.to_netcdf(save_dir + 'prop_data.nc')


sig90 = sel_sig_all(data_90, bootdata_90) # 90th Percentile


sig95 = sel_sig_all(data_95, bootdata_95) # 95th Percentile


prop_sig = xr.concat([sig90, sig95], pd.Index(['90','95'], name  = 'percentile'))

prop_sig.to_netcdf(save_dir + 'prop_sig.nc')

In [76]:
prop_sig


<xarray.Dataset>
Dimensions:     (lat: 53, lon: 178, mjo: 3, month: 6, percentile: 2)
Coordinates:
  * lat         (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
  * lon         (lon) float64 112.0 112.2 112.5 112.8 ... 155.8 156.0 156.2
  * month       (month) int64 10 11 12 1 2 3
  * mjo         (mjo) object 'enhanced' 'suppressed' 'inactive'
  * percentile  (percentile) object '90' '95'
Data variables:
    precip      (percentile, mjo, month, lat, lon) float32 nan nan ... nan nan

In [77]:
prop_data

<xarray.Dataset>
Dimensions:     (lat: 53, lon: 178, mjo: 3, month: 6, percentile: 2)
Coordinates:
  * lat         (lat) float64 -23.0 -22.75 -22.5 -22.25 ... -10.5 -10.25 -10.0
  * lon         (lon) float64 112.0 112.2 112.5 112.8 ... 155.8 156.0 156.2
  * month       (month) int64 10 11 12 1 2 3
  * mjo         (mjo) object 'enhanced' 'suppressed' 'inactive'
  * percentile  (percentile) object '90' '95'
Data variables:
    precip      (percentile, mjo, month, lat, lon) float32 nan nan ... nan nan